# Overview 
This notebook takes two repeat tracks in our study and samples Arctic DEM (2m) to estimate the topographic offset between repeats and investigate the corrlelation with our estimated elevation changes. In this notebook, we
- sample Arctic DEM at each point along track for each flyover date
- estimate the Arctic height difference  between the two dates for each point
- look at the correlation between the difference in Arctic DEM heights and IS2-estimated height change
- estimate the slope between the rpeat points based on the Arctic DEM heights
- calculate an adjusted IS2 height change estimate that removes the topographic offsets estimated from Arctic DEM

In [ ]:
# written by M.R. Siegfried
# 3 December 2020
#Last modified 15  July 2021 by Marnie Bryant
# siegfried@mines.edu
import pygmt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
import elevation
import richdem as rd
from pyproj import Proj, transform, Transformer

# Read in and format data

In [ ]:
# Set location of datafiles
dem_1280 = 'track_1280_2l_dem_2m.tif' #Arctic DEM subsetted to track domain and projected into ESPG 4326
dem_274 = 'track_274_1l_dem_2m.tif' #Arctic DEM subsetted to track domain and projected into ESPG 4326
is2='repeat_dh_filtered_slope_05_final.csv' #repeat track data

In [ ]:
# Read IS2 csv file and make new dataframes for each track
is2data=pd.read_csv(is2)
pt1=pd.DataFrame({'longitude': is2data['lon_1'], 
                  'latitude': is2data['lat_1']})

pt2=pd.DataFrame({'longitude': is2data['lon_2'], 
                  'latitude': is2data['lat_2']})

In [ ]:
#subset lines
is2_1280 = is2data[(is2data['rgt']==1280) & (is2data['spot_n']==3)].copy()
pt1_1280=pd.DataFrame({'longitude': is2_1280['lon_1'], 
                  'latitude': is2_1280['lat_1']})
pt2_1280=pd.DataFrame({'longitude': is2_1280['lon_2'], 
                  'latitude': is2_1280['lat_2']})


In [ ]:
is2_274 = is2data[(is2data['rgt']==274) & (is2data['spot_n']==1)].copy()
pt1_274=pd.DataFrame({'longitude': is2_274['lon_1'], 
                  'latitude': is2_274['lat_1']})
pt2_274=pd.DataFrame({'longitude': is2_274['lon_2'], 
                  'latitude': is2_274['lat_2']})

# DEM sampling

In [ ]:
# Sample 2m ArcticDEM at the points of each track - 1280 2l
print('sample ArcticDEM at lon_1,lat_1 (' + str(len(pt1_1280)) + ' points)')
pt1_1280=pygmt.grdtrack(points = pt1_1280, grid=dem_1280, newcolname='h_arcticdem', )
print('sample ArcticDEM at lon_2,lat_2 (' + str(len(pt2_1280)) + ' points)')
pt2_1280=pygmt.grdtrack(points = pt2_1280, grid=dem_1280, newcolname='h_arcticdem')
print('done sampling ArcticDEM')

In [ ]:
#add sampled points into dataframe
is2_1280['h1_arcticdem']=pt1_1280['h_arcticdem'].values
is2_1280['h2_arcticdem']=pt2_1280['h_arcticdem'].values
is2_1280['dh_arcticdem']=pt2_1280['h_arcticdem'].values - pt1_1280['h_arcticdem'].values


In [ ]:
# Sample 2m ArcticDEM at the points of each track - 274 1l
print('sample ArcticDEM at lon_1,lat_1 (' + str(len(pt1_274)) + ' points)')
pt1_274=pygmt.grdtrack(points = pt1_274, grid=dem_274, newcolname='h_arcticdem', )
print('sample ArcticDEM at lon_2,lat_2 (' + str(len(pt2_274)) + ' points)')
pt2_274=pygmt.grdtrack(points = pt2_274, grid=dem_274, newcolname='h_arcticdem')
print('done sampling ArcticDEM')

In [ ]:
#add sampled points to dataframe
is2_274['h1_arcticdem']=pt1_274['h_arcticdem'].values
is2_274['h2_arcticdem']=pt2_274['h_arcticdem'].values
is2_274['dh_arcticdem']=pt2_274['h_arcticdem'].values - pt1_274['h_arcticdem'].values


In [ ]:
#filter out large outliers
is2_1280_f = is2_1280[(is2_1280['dh_arcticdem'] <2)].copy()
#check how many points were removed
print('unfiltered: ', len(is2_1280), 'points')
print('filtered: ', len(is2_1280_f), 'points')

In [ ]:
is2_274_f = is2_274[(is2_274['dh_arcticdem'] <2)].copy()
print('unfiltered: ', len(is2_274), 'points')
print('filtered: ', len(is2_274_f), 'points')

# Correlation Analysis

In [ ]:
#correlation between dem dh and IS2 dh: 2m 1280 sampling
plt.plot(is2_1280_f['dh'],is2_1280_f['h2_arcticdem']-is2_1280_f['h1_arcticdem'],'.')
plt.title('1280 gt2l: 2m DEM')
plt.xlabel('is2 dh (m)')
plt.ylabel('DEM dh (m)')
#plt.ylim([-0.5, 0.5])
plt.show()
corr=is2_1280_f['dh'].corr(is2_1280_f['dh_arcticdem'])
print('correlation between dh_icesat2 & dh_arcticdem is ' + str(corr)  + ' (r-squared = ' + str(corr**2) + ')')

In [ ]:
#correlation between dem dh and IS2 dh: 2m 274 sampling
plt.plot(is2_274['dh'],is2_274['h2_arcticdem']-is2_274['h1_arcticdem'],'.')
plt.title('274 gt1l: 2m DEM')
plt.xlabel('is2 dh (m)')
plt.ylabel('DEM dh (m)')
plt.show()
corr=is2_274['dh'].corr(is2_274['dh_arcticdem'])
print('correlation between dh_icesat2 & dh_arcticdem is ' + str(corr) + ' (r-squared = ' + str(corr**2) + ')')

# Calculate DEM slope and adjusted IS2 estimates
The following steps were not ultimately included in the paper, but may prove useful for further analysis

In [ ]:
#calculate distance and DEM slopes between points
is2_1280_f['distance'] = np.sqrt(is2_1280_f['distance_x']**2 + is2_1280_f['distance_y']**2)
is2_1280_f['dem_slope'] = (is2_1280_f['dh_arcticdem']/is2_1280_f['distance'])
is2_274_f['distance'] = (np.sqrt(is2_274_f['distance_x']**2 + is2_274_f['distance_y']**2))
is2_274_f['dem_slope'] = (is2_274_f['dh_arcticdem'])/is2_274_f['distance']

In [ ]:
#adjust dh values using arctic dem difference
is2_1280_f['dh_minus_dem'] = is2_1280_f['dh'] - is2_1280_f['dh_arcticdem']
is2_274_f['dh_minus_dem'] = is2_274_f['dh'] - is2_274_f['dh_arcticdem']

In [ ]:
#save output
is2_1280_f.to_csv('repeat_dh_1280_gt2l_2m_dem_demo.csv')
is2_274_f.to_csv('repeat_dh_274_gt1l_2m_dem_demo.csv')